In [50]:
import pandas as pd

In [51]:
df = pd.read_csv('커피빈매장.csv')
df

,Name,Address,Tel
0,삼성봉은사거리점,"서울시 강남구 영동대로 607 1,2층",02-3443-5618
1,코엑스웨스틴서울점,서울시 강남구 봉은사로 524 코엑스 인터컨티넨탈아케이드,02-569-3103
2,삼성오크우드점,서울 강남구 테헤란로87길 46 지하 2층,02-3466-8507
3,삼성봉은사로점,서울시 강남구 봉은사로 628 엘슨빌딩1층,02-538-4523
4,도심공항타워점,서울시 강남구 테헤란로87길 36 공항타워 1층,02-2016-5330
...,...,...,...
208,강동경희대병원앞점,서울특별시 강동구 동남로 885 양지빌딩 1층,02-481-9938
209,새문안로점,"서울특별시 종로구 새문안로 91 고려빌딩 1,2층",02-739-5806
210,광화문역1번출구점,서울시 종로구 새문안로5가길 7 1층 일부,02-723-3360
211,성균관대법학관점,서울시 종로구 성균관로 25-2 성균관대학교 법학관 B2,02-740-1958


In [52]:
#위도, 경도

In [53]:
import requests
import numpy as np

def get_location(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+address
    headers = {'Authorization':'KakaoAK 314059ac0c339079dbba4a9b47f7bf23'}
    res = requests.get(url, headers=headers).json()
    
    if res['documents']:
        x = res['documents'][0]['x']
        y = res['documents'][0]['y']
    else:
        x = np.nan
        y = np.nan
    
    return x, y

In [54]:
x, y = get_location('서울시 강남구 영동대로 607')
x, y  #경도, 위도?

('127.059486615422', '37.5145473024381')

In [55]:
store = pd.read_csv('커피빈매장.csv')
store['X'] = ''
store['Y'] = ''
store.head(2)

,Name,Address,Tel,X,Y
0,삼성봉은사거리점,"서울시 강남구 영동대로 607 1,2층",02-3443-5618,,
1,코엑스웨스틴서울점,서울시 강남구 봉은사로 524 코엑스 인터컨티넨탈아케이드,02-569-3103,,


In [56]:
for i in store.index:
    address = store.loc[i, 'Address']
    x, y = get_location(address)
    store.loc[i, 'X'] = x
    store.loc[i, 'Y'] = y

In [57]:
store.isnull().sum()
store.dropna(inplace=True)

In [58]:
store.head()

,Name,Address,Tel,X,Y
1,코엑스웨스틴서울점,서울시 강남구 봉은사로 524 코엑스 인터컨티넨탈아케이드,02-569-3103,127.057115600619,37.5129534255276
2,삼성오크우드점,서울 강남구 테헤란로87길 46 지하 2층,02-3466-8507,127.058205573267,37.5109823973192
3,삼성봉은사로점,서울시 강남구 봉은사로 628 엘슨빌딩1층,02-538-4523,127.064164448221,37.5146694627537
4,도심공항타워점,서울시 강남구 테헤란로87길 36 공항타워 1층,02-2016-5330,127.058597240146,37.5102336492314
5,트레이드타워점,서울시 강남구 영동대로 511 트레이드타워 지하1층,02-803-9994,127.061023782931,37.510356849269


In [68]:
store.to_csv('커피빈맵.csv',index=False)

In [69]:
address = '마포구'
filt = store['Address'].str.contains(address)

In [70]:
df = store[filt]
df

,Unnamed: 0,Name,Address,Tel,X,Y
21,24,마포경찰공제회점,"서울시 마포구 마포대로 78 (도화동 566), 경찰공제회 자람빌딩 1층",02-704-6591,126.949788,37.541897
32,39,홍대역8번출구점,서울시 마포구 양화로18안길 7 2층,02-338-9463,126.925088,37.556828
33,40,홍대역점,서울시 마포구 홍익로 6길 26,02-333-4278,126.923267,37.555472
50,57,상암SBS프리즘타워점,서울 마포구 상암산로 82 SBS 프리즘 타워 1층 로비,02-562-9605,126.892774,37.579716
51,58,상암SBS미디어센터점,서울시 마포구 상암산로 82 SBS프리즘타워 1층,02-2039-9027,126.892774,37.579716
53,60,상암한국지역정보개발원점,서울시 마포구 성암로301 KLID한국지역정보개발원 1층,02-376-9162,126.887937,37.582469


In [89]:
import folium
import warnings
warnings.filterwarnings(action = 'ignore')
store = pd.read_csv('커피빈맵.csv')

def get_map(address):
    filt = store['Address'].str.contains(address)
    df = store[filt]

    if len(df.index)!=0:
        first = df.head(1)
        x = first['X']
        y = first['Y']
        map = folium.Map(location=[y, x], zoom_start=13)
        
        for idx, row in df.iterrows():
            name = row['Name']
            x = row['X']
            y = row['Y']

            folium.Marker(
                location=[y, x], 
                popup=folium.Popup(f'[{idx}] {name}', max_width=200),
                icon=folium.Icon(color='red', icon='star')).add_to(map)

        map.save(f'{address}.html')
    else:
        print('못찾음')

get_map('마포구')